In [2]:
!pip install keras-tuner

     |████████████████████████████████| 133 kB 4.1 MB/s 


In [1]:
from data_creation import DataSelection
from prediction_LSTM import SolarLSTM
import pickle

In [4]:
with open("/content/all_data.pkl", "rb") as file:
    data = pickle.load(file)

DataSelection(data, 2014, "./")
# DataSelection(data, 2015, "./")

Num GB used in data array: 0.0
Num GB used in data array: 0.0019025802612304688
Num GB used in data array: 0.0019025802612304688
Num GB used in data array: 0.0031256675720214844
Num GB used in data array: 0.0031256675720214844
Num GB used in data array: 0.010973811149597168
Num GB used in data array: 0.010973811149597168
Num GB used in data array: 0.01865208148956299
Num GB used in data array: 0.01865208148956299
Num GB used in data array: 0.02633035182952881
Num GB used in data array: 0.02633035182952881
Num GB used in data array: 0.03360092639923096
Num GB used in data array: 0.03360092639923096
Num GB used in data array: 0.035503506660461426
Num GB used in data array: 0.035503506660461426
Num GB used in data array: 0.037406086921691895
Num GB used in data array: 0.037406086921691895
Num GB used in data array: 0.04848182201385498
Num GB used in data array: 0.04848182201385498
Num GB used in data array: 0.04885554313659668
Num GB used in data array: 0.04885554313659668
Num GB used in 

In [2]:
import numpy as np

In [3]:
data2014 = np.load("/content/data_2014.npy")
labels2014 = np.load("/content/labels_2014.npy")

In [8]:
# data2014.shape[0] % 64

18

In [9]:
# # Need to adjust data size to ensure that each batch is of size 64
# adjusted_data = data2014[:data2014.shape[0]-18, :, :]
# adjusted_labels = labels2014[:labels2014.shape[0]-18, :, :]

In [4]:
base_model = SolarLSTM(data2014, labels2014, save_path="./")

In [5]:
base_model.build_model()

In [6]:
base_model.model

In [7]:
base_model.fit()

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


86/86 [==============================] - ETA: 0s - loss: 0.6677 - accuracy: 0.6070 - mse: 0.2373 - mae: 0.4731

INFO:tensorflow:Assets written to: model_checkpoints/assets


INFO:tensorflow:Assets written to: model_checkpoints/assets


86/86 [==============================] - 43s 468ms/step - loss: 0.6677 - accuracy: 0.6070 - mse: 0.2373 - mae: 0.4731 - val_loss: 0.6457 - val_accuracy: 0.6377 - val_mse: 0.2270 - val_mae: 0.4487
Epoch 2/50
86/86 [==============================] - 30s 344ms/step - loss: 0.6408 - accuracy: 0.6707 - mse: 0.2240 - mae: 0.4459 - val_loss: 0.6490 - val_accuracy: 0.6376 - val_mse: 0.2285 - val_mae: 0.4516
Epoch 3/50
86/86 [==============================] - 30s 344ms/step - loss: 0.6304 - accuracy: 0.6832 - mse: 0.2192 - mae: 0.4408 - val_loss: 0.6575 - val_accuracy: 0.6379 - val_mse: 0.2322 - val_mae: 0.4451
